In [1]:
import pandas as pd
import os
from glob import glob
import arcpy
from arcpy import env

In [2]:
yr = '2006'

wdir = 'F:/LinYuxuan/CHI_yr'
vwmatrix = pd.read_csv('%s/vulnerability.csv' % wdir)
vwmatrix.set_index('stressors', inplace=True)

In [3]:
os.mkdir("%s/HI_depth/%s" % (wdir, yr))

for h in ['surface', 'mesopelagic', 'bathypelagic', 'abyssopelagic']:
    os.mkdir("%s/HI_depth/%s/%s" % (wdir, yr, h))
    
    # list habitats files
    os.chdir("%s/hbt_depth/%s" % (wdir, h))
    habitats = sorted(glob('*.tif'))
    # list stressors files
    os.chdir("%s/stressors_depth/%s" % (wdir, yr))
    stressors = sorted(glob('*.tif'))    
    
    env.workspace = "%s/hbt_depth/%s" % (wdir, h)
    
    for sts in stressors:
        try:
            sts_name = sts[0 : len(sts) - 18]
            sts_path = "%s/stressors_depth/%s/%s" % (wdir, yr, sts)
            num_of_factors = len(habitats)

            expression = ''
            rasters = []
            input_names = []

            for hbt in habitats:
                hbt_name = hbt[len(h) + 1 : len(hbt) - 4]
                factor = str(vwmatrix.loc[sts_name, hbt_name])
                expression = expression + hbt_name + '*' + factor + '+'
                rasters.append(hbt)
                input_names.append(hbt_name)

            expression = expression[0 : len(expression) - 1]
            rasters.append(sts_path)
            input_names.append(sts_name)
            expression = ' ( %s ) * %s / %s' % (expression, sts_name, num_of_factors)

            output_raster = arcpy.ia.RasterCalculator(rasters, input_names, expression); 
            output_raster.save("%s/HI_depth/%s/%s/HI_%s" % (wdir, yr, h, sts))
            print("%s: %s --- complete" % (h, sts_name))
        except:
            print("====== error: %s: %s ======" % (h, sts_name))
            with open("%s/HI_depth/log_%s_%s.txt" % (wdir, yr, h), 'a') as t:
                t.write(sts_name + '\n' + expression + '\n' + rasters + '\n' + input_names + '\n\n' )
        

====== error: surface: artisanal_fishing ======


TypeError: can only concatenate str (not "list") to str